In [6]:
# get all the name from dir results_private_test

import os 
import json

def get_name_from_dir(dir_path):
    names = []
    for file in os.listdir(dir_path):
        if file.endswith(".json"):
            names.append(file.split(".")[0])
    return names


In [8]:
private_results = get_name_from_dir('result_private_test')

In [151]:
private_results

['private_vintern_v1_1_epoch4',
 'private_vintern_v1_1_epoch5',
 'private_vintern_v1_1_epoch6',
 'private_vintern_v1_1_epoch7',
 'private_vintern_v1_epoch3',
 'private_vintern_v1_epoch5',
 'private_vintern_v2_epoch2',
 'Vintern_private_yes_no_ova_v1_epoch2']

In [9]:
import json
def read_submission(name_file):
    # json file
    with open(name_file, 'r') as f:
        data = json.load(f)
    return list(data["results"].values())


In [10]:
import pandas as pd

def get_all_results(names):
    all_results = []
    for name in names:
        results = read_submission(f"result_private_test/{name}.json")
        all_results.append(results)
    return all_results

all_results = get_all_results(private_results)


In [14]:
def count_sarcasm(file_name):
    with open(file_name, 'r') as f:
        data = json.load(f)
    count_map = {
        "not-sarcasm": 0,
        "multi-sarcasm": 0,
        "text-sarcasm": 0,
        "image-sarcasm": 0
    }
    for label in data["results"].values():
        count_map[label] += 1
    return count_map

In [17]:
private_results

['private_vintern_v1_1_epoch4',
 'private_vintern_v1_1_epoch5',
 'private_vintern_v1_1_epoch6',
 'private_vintern_v1_1_epoch7',
 'private_vintern_v1_epoch3',
 'private_vintern_v1_epoch5',
 'private_vintern_v2_epoch2',
 'Vintern_private_yes_no_ova_v1_epoch2']

In [18]:
import pandas as pd

# create a table with the columns is name of the file and the col is 4 sarcasm type

def create_table(names, all_results):
    data = []
    for i in range(len(names)):
        count_map = count_sarcasm(f"result_private_test/{names[i]}.json")
        data.append([names[i], count_map["not-sarcasm"], count_map["multi-sarcasm"], count_map["text-sarcasm"], count_map["image-sarcasm"]])
    return pd.DataFrame(data, columns=["name", "not-sarcasm", "multi-sarcasm", "text-sarcasm", "image-sarcasm"])

In [20]:
create_table(private_results[:-1], all_results)

,name,not-sarcasm,multi-sarcasm,text-sarcasm,image-sarcasm
0,private_vintern_v1_1_epoch4,553,936,0,15
1,private_vintern_v1_1_epoch5,570,912,0,22
2,private_vintern_v1_1_epoch6,581,899,0,24
3,private_vintern_v1_1_epoch7,583,893,0,28
4,private_vintern_v1_epoch3,523,981,0,0
5,private_vintern_v1_epoch5,575,892,0,37
6,private_vintern_v2_epoch2,545,959,0,0


In [21]:
def transform_submit_2(best_result, best_result_2, best_result_3, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        label1 = best_result[i]
        label2 = best_result_2[i]
        label3 = best_result_3[i]
        yes_no_label = yes_no[i]

        # Initialize the transformed label with the first result as default
        transformed_label = label1

        # Conditions for 'multi-sarcasm'
        if (
            # not-sarcasm not-sarcasm not-sarcasm Yes: 48
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm multi-sarcasm multi-sarcasm Yes: 30
            (label1 == 'not-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm not-sarcasm multi-sarcasm No: 6
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'No') or
            # not-sarcasm multi-sarcasm not-sarcasm Yes: 18
            (label1 == 'not-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm multi-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm multi-sarcasm image-sarcasm No: 1
            (label1 == 'not-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'No') or
            # not-sarcasm not-sarcasm image-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm not-sarcasm multi-sarcasm Yes: 3
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm not-sarcasm Yes: 1
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'Yes')
        ):
            transformed_label = 'multi-sarcasm'

        # Conditions for 'not-sarcasm'
        elif (
            # multi-sarcasm multi-sarcasm multi-sarcasm No: 19
            (label1 == 'multi-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'No') or
            # multi-sarcasm not-sarcasm not-sarcasm No: 17
            (label1 == 'multi-sarcasm' and label2 == 'not-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'No') or
            # image-sarcasm multi-sarcasm multi-sarcasm Yes: 2
            (label1 == 'image-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes')
        ):
            transformed_label = 'not-sarcasm'

        # Conditions for 'image-sarcasm'
        elif (
            # image-sarcasm image-sarcasm image-sarcasm Yes: 6
            (label1 == 'image-sarcasm' and label2 == 'image-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm image-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # multi-sarcasm multi-sarcasm not-sarcasm No: 1
            (label1 == 'multi-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'No')
        ):
            transformed_label = 'image-sarcasm'

        # Append the transformed label
        transformed_labels.append(transformed_label)

    return transformed_labels

In [66]:
def transform_submit(best_result, best_result_2, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "not-sarcasm" and best_result_2[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
        elif best_result[i] == "not-sarcasm" and best_result_2[i] == "multi-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
        elif best_result[i] == "multi-sarcasm" and best_result_2[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and best_result_2[i] == "image-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "image-sarcasm"
        elif best_result[i] == "multi-sarcasm" and best_result_2[i] == "image-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "image-sarcasm"
    return transformed_labels

In [118]:
def old_transform_submit(best_result, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
    return transformed_labels

In [ ]:
# vintern_v1_1_epoch6

name_best_result = "private_vintern_v1_1_epoch6"
name_best_results_2 = "private_vintern_v1_epoch5"
name_best_results_3 = "private_vintern_v1_1_epoch7"
name_yes_no = "Vintern_private_yes_no_ova_v1_epoch2"

with open(f"result_private_test/{name_best_result}.json", 'r') as f:
    data = json.load(f)
    best_result = list(data["results"].values())
    
with open(f"result_private_test/{name_best_results_2}.json", 'r') as f:
    data = json.load(f)
    best_results_2 = list(data["results"].values())

with open(f"result_private_test/{name_best_results_3}.json", 'r') as f:
    data = json.load(f)
    best_results_3 = list(data["results"].values())
    
with open(f"result_private_test/{name_yes_no}.json", 'r') as f:
    data = json.load(f)
    yes_no = list(data["results"].values())



In [90]:
final_result = transform_submit_2(best_result, best_results_2, best_results_3, yes_no)
# count final results 
from collections import Counter
Counter(final_result)

Counter({'multi-sarcasm': 1012, 'not-sarcasm': 469, 'image-sarcasm': 23})

In [ ]:
final_result = transform_submit(best_result, best_results_2, yes_no)
# count final results 
from collections import Counter
Counter(final_result)

Counter({'multi-sarcasm': 1017, 'not-sarcasm': 455, 'image-sarcasm': 32})

In [137]:
final_result = old_transform_submit(best_result, yes_no)
# count final results 
from collections import Counter
Counter(final_result)

Counter({'multi-sarcasm': 999, 'not-sarcasm': 481, 'image-sarcasm': 24})

In [138]:
results = {}
results["results"] = {}
results["phase"] = "test"

for i in range(len(final_result)):
    results["results"][str(i)] = final_result[i]
with open(f"submissions_private_test/val_best_{name_best_result}_not_{name_yes_no}_results.json", "w") as out:
    json.dump(results, out)

In [150]:
with open("submissions_private_test/val_best_private_vintern_v1_1_epoch6_not_Vintern_private_yes_no_ova_v1_epoch2_results.json") as f:
    data = json.load(f)
    
Counter(list(data["results"].values()))

Counter({'multi-sarcasm': 999, 'not-sarcasm': 481, 'image-sarcasm': 24})